# Trying to find the source of my error

This notebook aims to test the features of the line-of-sight branch of lenstronomy, which includes the SinglePlaneLOS() class and the two line-of-sight models: 'LOS' and 'LOS_MINIMAL'. In particular, we check that a lens with line-of-sight corrections is equivalent to a multi-plane lens setting.

<div class="alert alert-block alert-info">
    
Importantly, in the EPL + 3 shear planes defined above, there are non-linear couplings between the shear planes which lead effectively to second-order convergence and rotation terms in the line-of-sight corrections. These must be taken into account.
    
Let's call $\boldsymbol{\Gamma}_f, \boldsymbol{\Gamma}_d, \boldsymbol{\Gamma}_b$ the three shear matrices in the foreground, dominant, and background planes. The exact displacement angle in that case reads
$$
\boldsymbol{\alpha}(\boldsymbol{\theta})
=
\boldsymbol{\Gamma}_{os}\boldsymbol{\theta}
+ (1-\boldsymbol{\Gamma}_{ds})
    \boldsymbol{\alpha}_{ods}[(1-\boldsymbol{\Gamma}_{od})\boldsymbol{\theta}]
$$
with
\begin{align}
\boldsymbol{\Gamma}_{od}
&= \boldsymbol{\Gamma}_{ofd}
= \frac{D_{fd}D_{os}}{D_{od}D_{fs}} \boldsymbol{\Gamma}_{f}
\\
\boldsymbol{\Gamma}_{ds}
&= \boldsymbol{\Gamma}_{dbs}
= \frac{D_{os}D_{db}}{D_{ob}D_{ds}} \boldsymbol{\Gamma}_{f}
\\
\boldsymbol{\Gamma}_{os}
&= \boldsymbol{\Gamma}_{f} + \boldsymbol{\Gamma}_{d} + \boldsymbol{\Gamma}_{b}
    - \left[
        \boldsymbol{\Gamma}_{d} \boldsymbol{\Gamma}_{od}
        + \boldsymbol{\Gamma}_{b} \boldsymbol{\Gamma}_{ofb}
        + \boldsymbol{\Gamma}_{b} \boldsymbol{\Gamma}_{ofb} (1-\boldsymbol{\Gamma}_{od})
        \right]
\end{align}

## 1. Define the lens models

In [3]:
# Import of standard python libraries
import numpy as np
import os
import time
import corner
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from ipywidgets import interact
%matplotlib inline

Define a lens with line-of-sight shear using the standard approach of lenstronomy, i.e. add 3 shear planes to the main lens, here modelled as an elliptical power law (EPL).

In [10]:
lens_model_list_shears = ['EPL', 'SHEAR', 'SHEAR', 'SHEAR']
lens_model_list_los = ['EPL', 'LOS']

# Parameters of the main lens
# We define its centre as the line of sight
kwargs_epl = {'theta_E': 0.8,
               'gamma': 1.95,
               'center_x': 0,
               'center_y': 0,
               'e1': .07,
               'e2': -0.03}

# Cosmology (we need it for the distances that are involved in the relations between shears above)
from astropy.cosmology import default_cosmology
cosmo = default_cosmology.get()

# Lensing setup to generate the 'true' (mock) image:
# a main lens (EPL) and three shear planes
z_s = 2
z_d = 0.5 # redshift of dominant lens
z_f = z_d/2 # redshift of foreground shear plane
z_b = (z_s + z_d)/2 # redshift of background shear plane
lens_redshift_list_shear = [z_d, z_f, z_d, z_b]

def function(gamma_od_1,gamma_od_2,gamma_os_1,gamma_os_2,gamma_ds_1,gamma_ds_2,x):
    ##Shear Model

    # LOS (linear) shears
    gamma_od = np.array([gamma_od_1, gamma_od_2])
    gamma_os = np.array([gamma_os_1, gamma_os_2])
    gamma_ds = np.array([gamma_ds_1, gamma_ds_2])

    # Convert line-of-sight shears into lenstronomy shears
    gamma_f = gamma_od*(
        cosmo.angular_diameter_distance_z1z2(0, z_d)
        * cosmo.angular_diameter_distance_z1z2(z_f, z_s)
        / cosmo.angular_diameter_distance_z1z2(0, z_s)
        / cosmo.angular_diameter_distance_z1z2(z_f, z_d))
    gamma_b = gamma_ds*(
        cosmo.angular_diameter_distance_z1z2(0, z_b)
        * cosmo.angular_diameter_distance_z1z2(z_d, z_s)
        / cosmo.angular_diameter_distance_z1z2(0, z_s)
        / cosmo.angular_diameter_distance_z1z2(z_d, z_b))
    gamma_d = gamma_os - gamma_f - gamma_b
           
    kwargs_gamma_f = {'gamma1': float(gamma_f[0]), 'gamma2': float(gamma_f[1])}
    kwargs_gamma_d = {'gamma1': float(gamma_d[0]), 'gamma2': float(gamma_d[1])}
    kwargs_gamma_b = {'gamma1': float(gamma_b[0]), 'gamma2': float(gamma_b[1])}
    # I used float() here because otherwise astropy complains about units

    # Generate the complete lens model
    kwargs_lens_shears = [kwargs_epl,
                          kwargs_gamma_f,
                          kwargs_gamma_d,
                          kwargs_gamma_b]
    from lenstronomy.LensModel.lens_model import LensModel
    lens_model_class_shears = LensModel(
        lens_model_list_shears,
        lens_redshift_list=lens_redshift_list_shear,
        z_lens=z_d,
        z_source=z_s,
        multi_plane=True)
    
    ##LOS model
    
    # The (od) and (ds) corrections are the ones that we have entered first:
    kwargs_los = {
        'kappa_od': 0, 'kappa_ds': 0,
        'omega_od': 0, 'omega_ds': 0,
        'gamma1_od': gamma_od[0], 'gamma2_od': gamma_od[1],
        'gamma1_ds': gamma_ds[0], 'gamma2_ds': gamma_ds[1]}

    # but the (os) perturbations are affected by non-linear couplings
    Id = np.array([[1, 0],
                   [0, 1]])
    Gamma_f = np.array([[float(gamma_f[0]), float(gamma_f[1])],
                        [float(gamma_f[1]), -float(gamma_f[0])]])
    Gamma_d = np.array([[float(gamma_d[0]), float(gamma_d[1])],
                        [float(gamma_d[1]), -float(gamma_d[0])]])
    Gamma_b = np.array([[float(gamma_b[0]), float(gamma_b[1])],
                        [float(gamma_b[1]), -float(gamma_b[0])]])
    Gamma_od = np.array([[gamma_od[0], gamma_od[1]],
                         [gamma_od[1], -gamma_od[0]]])
    Gamma_ofb = float(cosmo.angular_diameter_distance_z1z2(0, z_s)
                     * cosmo.angular_diameter_distance_z1z2(z_f, z_b)
                     / cosmo.angular_diameter_distance_z1z2(0, z_b)
                     / cosmo.angular_diameter_distance_z1z2(z_f, z_s)) * np.array(Gamma_f)
    Gamma_odb = float(cosmo.angular_diameter_distance_z1z2(0, z_s)
                     * cosmo.angular_diameter_distance_z1z2(z_d, z_b)
                     / cosmo.angular_diameter_distance_z1z2(0, z_b)
                     / cosmo.angular_diameter_distance_z1z2(z_d, z_s)) * np.array(Gamma_d)

    Gamma_os = (Gamma_f + Gamma_d + Gamma_b
                - np.matmul(Gamma_d, Gamma_od)
                - np.matmul(Gamma_b, Gamma_ofb + np.matmul(Gamma_odb, Id - Gamma_od))
               )

    kappa_os = (Gamma_os[0,0] + Gamma_os[1,1]) / 2
    gamma1_os = (Gamma_os[0,0] - Gamma_os[1,1]) / 2
    gamma2_os = (Gamma_os[0,1] + Gamma_os[1,0]) / 2
    omega_os = (Gamma_os[1,0] - Gamma_os[0,1]) / 2

    kwargs_los.update({'kappa_os': 0, 'omega_os': 0,
                       'gamma1_os': float(gamma_os[0]), 'gamma2_os': float(gamma_os[1])})

    kwargs_los.update({'kappa_os': kappa_os, 'omega_os': omega_os,
                       'gamma1_os': gamma1_os, 'gamma2_os': gamma2_os})

    # Generate the full model
    kwargs_lens_los = [kwargs_epl, kwargs_los]
    from lenstronomy.LensModel.lens_model import LensModel
    lens_model_class_los = LensModel(lens_model_list_los,
        z_lens=z_d,
        z_source=z_s)    
                         
    y = np.linspace(-5,5,100)
    xx = x*np.ones(100)
                         
    phi_shears = lens_model_class_shears.fermat_potential(xx, y, kwargs_lens_shears)
    phi_los = lens_model_class_los.fermat_potential(xx, y, kwargs_lens_los)
    
    plt.figure()
    line1=plt.plot(y,phi_los,label='LOS Fermat Potential')
    line2=plt.plot(y,phi_shears,label='Multi-Plane Fermat Potential')      
    plt.legend()
    plt.xlabel("y")
    plt.ylabel("Fermat Potential")
    
interact(function, gamma_od_1=(-0.9,0.9,0.02),gamma_od_2=(-0.9,0.9,0.02),gamma_os_1=(-0.9,0.9,0.02),gamma_os_2=(-0.9,0.9,0.02),gamma_ds_1=(-0.9,0.9,0.02),gamma_ds_2=(-0.9,0.9,0.02),x=(-5,5,0.2))     

interactive(children=(FloatSlider(value=0.0, description='gamma_od_1', max=0.9, min=-0.9, step=0.02), FloatSli…

<function __main__.function(gamma_od_1, gamma_od_2, gamma_os_1, gamma_os_2, gamma_ds_1, gamma_ds_2, x)>